In [73]:
import pandas as pd
import numpy as np

In [74]:
# Load the adult BPM dataset
bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Bradycardia/CSV_DATA/BPM_8hr_5min.csv"
df = pd.read_csv(bpm_file, header=None)  # Assuming no headers in file

In [75]:
# Define adult and neonatal BPM ranges
adult_normal_range = (60, 100)
adult_brady_range = (df.min().min(), 60)  # Anything <60 is bradycardia
neonatal_normal_range = (100, 160)  # Shift lower to allow more bradycardia
neonatal_brady_range = (60, 90)  # Increase bradycardia severity

In [76]:
# Function to rescale values with enhanced bradycardia effect
def rescale_bpm(data, old_min, old_max, new_min, new_max):
    return ((data - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

In [77]:
# Apply rescaling
df_neonatal = df.copy()
# Scale normal BPM values (60-100) -> Neonatal normal range (100-160)
df_neonatal[df >= 60] = rescale_bpm(df[df >= 60], *adult_normal_range, *neonatal_normal_range)

# Scale bradycardia BPM values (<60) -> Neonatal bradycardia range (30-85)
df_neonatal[df < 60] = rescale_bpm(df[df < 60], *adult_brady_range, *neonatal_brady_range)

In [78]:
# Introduce slight random variation to bradycardia values to prevent uniform scaling
brady_mask = df_neonatal < 85
df_neonatal[brady_mask] += np.random.uniform(-5, 5, size=df_neonatal[brady_mask].shape)

In [79]:
# Save the rescaled neonatal dataset
neonatal_bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Bradycardia/Scaled_CSV_DATA/BPM_8hr_5min_Neonatal.csv"
df_neonatal.to_csv(neonatal_bpm_file, index=False)

print(f"Neonatal BPM data saved: {neonatal_bpm_file}")

Neonatal BPM data saved: /home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Bradycardia/Scaled_CSV_DATA/BPM_8hr_5min_Neonatal.csv


Printing instances of bradycardia and severe bradycardia in the neonatal dataset:

In [80]:
import pandas as pd

# Load the neonatal BPM dataset
neonatal_bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Bradycardia/Scaled_CSV_DATA/BPM_8hr_5min_Neonatal.csv"
df_neonatal = pd.read_csv(neonatal_bpm_file, header=None)

# Remove the first row if it contains index-like values
if (df_neonatal.iloc[0] == range(96)).all():  # Check if first row is just index values
    df_neonatal = df_neonatal.iloc[1:].reset_index(drop=True)  # Remove it

# Convert to numeric to ensure proper processing
df_neonatal = df_neonatal.apply(pd.to_numeric, errors="coerce")

# Define neonatal bradycardia thresholds
brady_threshold = 100
severe_brady_threshold = 80

# Identify bradycardia instances (BPM < 100)
bradycardia_instances = df_neonatal[df_neonatal < brady_threshold].stack().reset_index()
bradycardia_instances.columns = ["Row", "Column", "BPM"]

# Identify severe bradycardia instances (BPM < 80)
severe_brady_instances = bradycardia_instances[bradycardia_instances["BPM"] < severe_brady_threshold]

# Print the detected bradycardia instances
print("\nBradycardia Instances (BPM < 100):")
print(bradycardia_instances)

print("\nSevere Bradycardia Instances (BPM < 80):")
print(severe_brady_instances)



Bradycardia Instances (BPM < 100):
    Row  Column        BPM
0     3       7  76.198411
1     3      13  79.590951
2     3      78  89.850307
3     5      23  78.066394
4     5      30  83.448040
5     5      32  73.378475
6     5      33  81.898846
7     5      34  68.770283
8     5      45  70.103878
9     5      49  74.609193
10    5      57  79.021338
11    5      58  77.006583
12    5      62  74.792954
13    5      81  88.515411
14    5      84  89.814712
15    5      85  66.920340
16    5      86  63.614435
17    5      88  74.325567
18    9      10  85.996606
19    9      23  69.833752
20    9      30  68.075274
21    9      32  69.210798
22    9      33  65.228224
23    9      34  68.799676
24    9      35  88.570799
25    9      45  68.701754
26    9      49  74.966866
27    9      57  84.708231
28    9      58  77.560152
29    9      61  76.360744
30    9      62  71.790191
31    9      82  85.704802
32    9      85  60.252976
33    9      86  57.223835
34    9      88  82